## Import Libraries

In [ ]:
import os
import numpy as np
import loompy as lp
import datetime
from itertools import count as iter_count

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

# from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras import regularizers
# from keras.layers import Conv2D, MaxPooling2D

from keras.layers import Dense,  Dropout, Activation, BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras.losses import mean_squared_error, mean_absolute_error
from keras.activations import relu, elu, linear
from keras import backend as K

from hyperas import optim
from hyperopt import Trials, STATUS_OK, tpe
from hyperas.distributions import choice, uniform, conditional

import matplotlib.pyplot as plt

## Miscellaneous parameters

In [ ]:
# Create train, evaluation and test ratios for splitting the dataset 
TRAIN_RT = 0.70
EVAL_RT = 0.15
TEST_RT = 0.15

## External Custom Functions

### 1) Split the dataset in train, test & eval sets (index based split)
This function returns indices for the train, test and evaluation sets, given an input Dataset.

Arguments:

    train_rt: ratio of the train dataset
    test_rt:  ratio of the test dataset
    eval_rt:  ratio of the evaluation dataset

Returns:

    train_idx: indices (of the given dataset) for the train dataset
    test_idx:  indices (of the given dataset) for the test dataset
    evel_idx:  indices (of the given dataset) for the evaluation dataset

Note:

    This function will work correctly as long as (test_rt == evel_rt) is True.
    If you need (test_rt != evel_rt), you need something more sophisticated.


In [ ]:
def train_test_set_idx_split(train_rt, test_rt, eval_rt):
    with lp.connect('/proj/notebooks/data_QC_min250_GpC_f.loom', 'r') as ds:
        idx = np.array(range(0, ds.shape[1]))

    train_idx, test_idx = train_test_split(idx, train_size=train_rt, test_size=test_rt+eval_rt, random_state=0)
    test_idx, eval_idx = train_test_split(test_idx, train_size=0.5, test_size=0.5, random_state=0)

    return train_idx, test_idx, eval_idx

### 2) Coefficient of Determination R2
The coefficient of determination R2 can describe how “good” a model is at making predictions. It represents the proportion of the variance in the dependent variable that is predictable from the independent variable,

where:

    SS_tot: is the total sum of squares,
    SS_res: is the residual sum of squares, and is the predicted value.

R2 ranges from 0 to 1:

    if R2=0 : the model always fails to predict the target variable,
    if R2=1 : the model perfectly predicts the target variable.
    
Any value between 0 and 1 indicates what percentage of the target variable, using the model, can be explained by the features. If R2 < 0, it indicates that the model is no better than one that constantly predicts the mean of the target variable.

In [ ]:
def coeff_determ_r2(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return (1 - SS_res / (SS_tot + K.epsilon()))

### 3) Data Generator
Yields batch features and batch labels.

`reset_data()` function, gets indices as an argument, shuffles them and returns them along with an new infinite iterator (starting from 0 with step 1).

In [ ]:
def reset_data(data):
    data = shuffle(data)
    inf_iterator = iter_count(start = 0, step = 1)
    
    return inf_iterator, data

def generator(batch_size, mode):
    with lp.connect('/proj/notebooks/data_QC_min250_GpC_f.loom', 'r') as ds:
        
        train_idx, test_idx, eval_idx = train_test_set_idx_split(0.7, 0.15, 0.15)
        
        if mode == 'TRAIN':
            indices = train_idx
            inf_iterator, indices = reset_data(indices)
        elif mode == 'EVAL':
            indices = eval_idx
            inf_iterator, indices = reset_data(indices)
        elif mode == 'TEST':
            indices = test_idx
            inf_iterator = iter_count(start = 0, step = 1)
        else:
            print("Wrong mode choice: ", mode)
            exit(1)

        while True:
            batch_count = next(inf_iterator)
            batch_indices = np.sort(indices[batch_count*batch_size : batch_count*batch_size + batch_size])
            
            if (mode == 'TRAIN' and batch_count == 1630) or (mode == 'EVAL' and batch_count == 299):
                inf_iterator, indices = reset_data(indices)
            elif (mode == 'TEST' and batch_count == 69 * 2 - 1):
                inf_iterator = iter_count(start = 0, step = 1)
            else:
                pass

            try:
                batch_features = ds[:, batch_indices].T
                batch_labels = ds.ca.Age[batch_indices]
#                 print(batch_labels, batch_count, batch_indices)
            except Exception as e:
                print("batch_count = {0} - mode: {1} - idx: {2}".format(batch_count, mode, batch_indices))
                print("Error: ", e)

            yield batch_features, batch_labels

### 4) Test Generator
Yields non-shuffled test data and their labels.

In [ ]:
def test_generator():
    with lp.connect('/proj/notebooks/data_QC_min250_GpC_f.loom', 'r') as ds:

        _, test_idx, _ = train_test_set_idx_split(TRAIN_RT, TEST_RT, EVAL_RT)
        idx = np.sort(test_idx)
        count = 0
      
        while True:
            # for i in range(batch_size):    ##### Think about total num of cells / batch size
            print(count)
            count = count + 1
            # print(ds.ca.Age[idx])

            yield ds[:, idx].T, ds.ca.Age[idx]

### 5) Data Function
This function provides the train, validation and predict generators for the model

In [ ]:
def data():
    train_generator = generator(69, mode = 'TRAIN')
    validation_generator = generator(69, mode = 'EVAL')
    predict_generator = generator(69, mode = 'TEST')
    
    return train_generator, validation_generator, predict_generator

## Declare Model

#### Some fixed parameters are based on previous Hyperparameter optimization with Hyperas
You may need to change some of the parameters to adapt to your needs/data

In [ ]:
def model(train_generator, validation_generator):

    with lp.connect('/proj/notebooks/data_QC_min250_GpC_f.loom', 'r') as ds:
        input_size = ds.shape[0]
    l1 = regularizers.l1

    model = Sequential()
    
    model.add(Dropout({{uniform(0, 1)}}, input_shape=(input_size,)))

    # Layer 1
    model.add(Dense(1000, activation=None))
    model.add(BatchNormalization())
    model.add(Activation('elu'))

    # Layer 2
    model.add(Dense(1500, activation='relu', activity_regularizer=l1(10e-6)))
    model.add(Dropout({{uniform(0, 1)}}))

    # Layer 3
    model.add(Dense({{choice([500, 200, 100, 50, 10])}}, activation=None))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    # Layer 4
    model.add(Dense({{choice([500, 200, 100, 50, 10])}}, activation='relu', activity_regularizer=l1(10e-6)))

    # Last Layer
    model.add(Dense(1,  activation={{choice(['linear', 'elu', 'relu', None])}}))
    
    model.compile(optimizer='nadam', loss='logcosh', metrics=['mse', coeff_determ_r2])
    
#     earlystopping = EarlyStopping(monitor='val_loss', mode = 'min')
#     tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0)
#     checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
#     checkpoint = ModelCheckpoint('./mycheckpoints/' + checkpoint_name, monitor='val_loss', verbose = 1, 
#                              save_best_only = True, mode ='auto')

#                         callbacks=[earlystopping, tensorboard, checkpoint],
    
    hist = model.fit_generator(generator=train_generator,
                                epochs = 10,
                                steps_per_epoch = 1631,
                                validation_data = validation_generator,
                                validation_steps= 300,
                                workers=1,
                                use_multiprocessing=False)
        
    metrics = model.evaluate_generator(generator=validation_generator, steps= 100)
    
    return {'loss': -metrics[2], 'mse': metrics[1], 'l': metrics[0], 'status': STATUS_OK, 'model': model}

In [ ]:
# Define the functions to be used by the model
functions=[generator, coeff_determ_r2, reset_data, train_test_set_idx_split]

# Get the optimization results
best_run, best_model = optim.minimize(model=model,
                                      data=data,
                                      functions=functions,
                                      algo=tpe.suggest,
                                      max_evals=5,
                                      trials=Trials(),
                                      notebook_name='Optimized_lvl-3ugh')

## Results

In [ ]:
# Print the best run parameter
print(best_run)

In [ ]:
# Run Validation

_, validation_generator, _ = data()
print("Evaluation of best performing model:")
metrics = best_model.evaluate_generator(generator=validation_generator, 
                                      steps=1000)

In [ ]:
# Print validation result metrics 
metrics

## Save the model

In [ ]:
best_model.save('best_model-hpr_v003.h5')

In [ ]:
best_model.save_weights('best_model_weights-hpr_v003.h5')

## Plot predictions on test data

In [ ]:
# Load test data and labels
_, test_idx, _ = train_test_set_idx_split(TRAIN_RT, TEST_RT, EVAL_RT)

with lp.connect('/proj/notebooks/data_QC_min250_GpC_f.loom', 'r') as ds:
    age = ds.ca.Age[np.sort(test_idx)]

In [ ]:
# Load the model if needed
#new_model = load_model('best_model-hpr_v003.h5', custom_objects={'coeff_determ_r2': coeff_determ_r2})

In [ ]:
# Predict the test ages
test_gen = test_generator()
best_model.predict_generator(test_gen, steps = 10)

In [ ]:
# Histogram plot of predicted age vs real age for each cell in the test data
%matplotlib notebook
plt.hist(predict, color='r', bins='fd', label='Predicted Age')
plt.hist(age, color='g', bins='fd', alpha=0.5, label='Original Age')
plt.legend()
plt.xlabel('Age', fontsize=16)
plt.ylabel('# of cells', fontsize=16)